# MTA New York City Subway Average Weekday Ridership 2014 - 2019

Data visualization of ridership activity for all 473 stations in the New York City subway system prior to the COVID-19 pandemic. A special thanks to Colin Patrick Reid for his contribution of the code template in his article "Exploring and Visualizing Chicago Transit data using pandas and Bokeh — Part II (intro to Bokeh)" in Towards Data Science (https://towardsdatascience.com/exploring-and-visualizing-chicago-transit-data-using-pandas-and-bokeh-part-ii-intro-to-bokeh-5dca6c5ced10). MTA subway ridership by station data can be found here: https://new.mta.info/agency/new-york-city-transit/subway-bus-ridership-2019. The geospatial coordinates, or geometry, of the subway stations can be found here: https://data.cityofnewyork.us/Transportation/Subway-Stations/arq3-7z49.

In [1]:
#Importing packages
import pandas as pd
import numpy as np
#from bokeh.io import show, output_notebook
from bokeh.plotting import figure, show, output_notebook
#from bokeh.tile_providers import CARTODBPOSITRON
from bokeh.tile_providers import get_provider, Vendors, CARTODBPOSITRON
from pandas import Series, DataFrame


In [2]:
cd '/Users/aaronwoodward/Desktop/Datasets/Cities/NYC/MTA Subway Ridership'

/Users/aaronwoodward/Desktop/Datasets/Cities/NYC/MTA Subway Ridership


In [3]:
#Import ridership data
ridership_df = pd.read_csv('MTA_Subway_Weekday_Ridership_2019 - Sheet1.csv')
ridership_df.head()


,Station_ID,Station_Name,Line,Boro,2014,2015,2016,2017,2018,2019,2018-2019 Change,Unnamed: 11,2019 Rank
0,1,Astor Pl,6,Manhattan,"17,803","17,274","16,806","16,377","16,031","17,180","+1,149",+7.2%,84.0
1,2,Canal St,"J,N,Q,R,W,Z,6",Manhattan,"44,362","44,409","45,030","47,601","47,034","47,629",+595,+1.3%,16.0
2,3,50th St,1,Manhattan,"27,251","26,927","26,362","25,932","26,268","26,712",+444,+1.7%,39.0
3,4,Bergen St,"2,3",Brooklyn,"3,923","3,823","3,884","3,853","3,653","3,857",+204,+5.6%,334.0
4,5,Pennsylvania Ave,3,Brooklyn,"5,718","6,997","2,485","3,503","4,331","4,493",+162,+3.8%,310.0


In [4]:
#ridership_df.tail()

In [4]:
#ridership_df = ridership_df.dropna()
ridership_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 473 entries, 0 to 472
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Station_ID        473 non-null    int64  
 1   Station_Name      473 non-null    object 
 2   Line              432 non-null    object 
 3   Boro              432 non-null    object 
 4   2014              428 non-null    object 
 5   2015              429 non-null    object 
 6   2016              429 non-null    object 
 7   2017              432 non-null    object 
 8   2018              432 non-null    object 
 9   2019              432 non-null    object 
 10  2018-2019 Change  432 non-null    object 
 11  Unnamed: 11       432 non-null    object 
 12  2019 Rank         432 non-null    float64
dtypes: float64(1), int64(1), object(11)
memory usage: 48.2+ KB


In [5]:
ridership_df = ridership_df.rename(columns={'Unnamed: 11':'Pct_chg', 
                                            '2014':'y_2014',
                                            '2015':'y_2015',
                                            '2016':'y_2016',
                                            '2017':'y_2017',
                                            '2018':'y_2018',
                                            '2019':'y_2019',
                                            'Line':'Subway_lines',
                                            '2018-2019 Change':'Chg',
                                            '2019 Rank':'Rank_2019'
                                           })
ridership_df.head()

,Station_ID,Station_Name,Subway_lines,Boro,y_2014,y_2015,y_2016,y_2017,y_2018,y_2019,Chg,Pct_chg,Rank_2019
0,1,Astor Pl,6,Manhattan,"17,803","17,274","16,806","16,377","16,031","17,180","+1,149",+7.2%,84.0
1,2,Canal St,"J,N,Q,R,W,Z,6",Manhattan,"44,362","44,409","45,030","47,601","47,034","47,629",+595,+1.3%,16.0
2,3,50th St,1,Manhattan,"27,251","26,927","26,362","25,932","26,268","26,712",+444,+1.7%,39.0
3,4,Bergen St,"2,3",Brooklyn,"3,923","3,823","3,884","3,853","3,653","3,857",+204,+5.6%,334.0
4,5,Pennsylvania Ave,3,Brooklyn,"5,718","6,997","2,485","3,503","4,331","4,493",+162,+3.8%,310.0


In [6]:
ridership_df['y_2014'] = ridership_df.y_2014.str.replace(',', '')
ridership_df['y_2015'] = ridership_df.y_2015.str.replace(',', '')
ridership_df['y_2016'] = ridership_df.y_2016.str.replace(',', '')
ridership_df['y_2017'] = ridership_df.y_2017.str.replace(',', '')
ridership_df['y_2018'] = ridership_df.y_2018.str.replace(',', '')
ridership_df['y_2019'] = ridership_df.y_2019.str.replace(',', '')

ridership_df.head()

,Station_ID,Station_Name,Subway_lines,Boro,y_2014,y_2015,y_2016,y_2017,y_2018,y_2019,Chg,Pct_chg,Rank_2019
0,1,Astor Pl,6,Manhattan,17803,17274,16806,16377,16031,17180,"+1,149",+7.2%,84.0
1,2,Canal St,"J,N,Q,R,W,Z,6",Manhattan,44362,44409,45030,47601,47034,47629,+595,+1.3%,16.0
2,3,50th St,1,Manhattan,27251,26927,26362,25932,26268,26712,+444,+1.7%,39.0
3,4,Bergen St,"2,3",Brooklyn,3923,3823,3884,3853,3653,3857,+204,+5.6%,334.0
4,5,Pennsylvania Ave,3,Brooklyn,5718,6997,2485,3503,4331,4493,+162,+3.8%,310.0


In [7]:
ridership_df

,Station_ID,Station_Name,Subway_lines,Boro,y_2014,y_2015,y_2016,y_2017,y_2018,y_2019,Chg,Pct_chg,Rank_2019
0,1,Astor Pl,6,Manhattan,17803,17274,16806,16377,16031,17180,"+1,149",+7.2%,84.0
1,2,Canal St,"J,N,Q,R,W,Z,6",Manhattan,44362,44409,45030,47601,47034,47629,+595,+1.3%,16.0
2,3,50th St,1,Manhattan,27251,26927,26362,25932,26268,26712,+444,+1.7%,39.0
3,4,Bergen St,"2,3",Brooklyn,3923,3823,3884,3853,3653,3857,+204,+5.6%,334.0
4,5,Pennsylvania Ave,3,Brooklyn,5718,6997,2485,3503,4331,4493,+162,+3.8%,310.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
468,469,Coney Island - Stillwell Av,"D,F,N,Q",Brooklyn,13734,14073,14098,13542,13203,12819,-384,-2.9%,122.0
469,470,34th St - Hudson Yards,7,Manhattan,NaN,2064,8507,10082,10789,18875,"+8,086",+75.0%,74.0
470,641,72nd St,Q,Manhattan,NaN,NaN,NaN,28145,30253,31585,"+1,332",+4.4%,30.0
471,642,86th St,Q,Manhattan,NaN,NaN,NaN,23722,25455,26307,+852,+3.3%,42.0


In [8]:
ridership_df.fillna(value=0, axis=0, inplace=True)
ridership_df.tail()

,Station_ID,Station_Name,Subway_lines,Boro,y_2014,y_2015,y_2016,y_2017,y_2018,y_2019,Chg,Pct_chg,Rank_2019
468,469,Coney Island - Stillwell Av,"D,F,N,Q",Brooklyn,13734,14073,14098,13542,13203,12819,-384,-2.9%,122.0
469,470,34th St - Hudson Yards,7,Manhattan,0,2064,8507,10082,10789,18875,"+8,086",+75.0%,74.0
470,641,72nd St,Q,Manhattan,0,0,0,28145,30253,31585,"+1,332",+4.4%,30.0
471,642,86th St,Q,Manhattan,0,0,0,23722,25455,26307,+852,+3.3%,42.0
472,643,96th St,Q,Manhattan,0,0,0,17150,19318,19704,+386,+2.0%,72.0


In [9]:
pd.set_option('display.max_rows', None)

In [10]:
ridership_df = ridership_df[ridership_df['Subway_lines']!=0]

In [11]:
ridership_df

,Station_ID,Station_Name,Subway_lines,Boro,y_2014,y_2015,y_2016,y_2017,y_2018,y_2019,Chg,Pct_chg,Rank_2019
0,1,Astor Pl,6,Manhattan,17803,17274,16806,16377,16031,17180,"+1,149",+7.2%,84.0
1,2,Canal St,"J,N,Q,R,W,Z,6",Manhattan,44362,44409,45030,47601,47034,47629,+595,+1.3%,16.0
2,3,50th St,1,Manhattan,27251,26927,26362,25932,26268,26712,+444,+1.7%,39.0
3,4,Bergen St,"2,3",Brooklyn,3923,3823,3884,3853,3653,3857,+204,+5.6%,334.0
4,5,Pennsylvania Ave,3,Brooklyn,5718,6997,2485,3503,4331,4493,+162,+3.8%,310.0
5,6,238th St,1,Bronx,3934,4068,4350,4370,3225,3998,+773,+24.0%,329.0
6,7,Cathedral Pkwy (110th St),1,Manhattan,13630,13444,12926,12394,12526,11826,-700,-5.6%,133.0
7,8,Kingston - Throop Aves,C,Brooklyn,5968,6210,6350,6564,6585,6643,+58,+0.9%,233.0
8,9,65th St,"M,R",Queens,3647,3762,3747,3621,3601,3554,-47,-1.3%,348.0
9,10,36th St,"M,R",Queens,4597,4733,4776,4786,5177,4725,-452,-8.7%,300.0


In [12]:
ridership_df['y_2014'] = ridership_df.y_2014.astype(int)
ridership_df['y_2015'] = ridership_df.y_2015.astype(int)
ridership_df['y_2016'] = ridership_df.y_2016.astype(int)
ridership_df['y_2017'] = ridership_df.y_2017.astype(int)
ridership_df['y_2018'] = ridership_df.y_2018.astype(int)
ridership_df['y_2019'] = ridership_df.y_2019.astype(int)
ridership_df['2019_Rank'] = ridership_df.Rank_2019.astype(int)

ridership_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 432 entries, 0 to 472
Data columns (total 14 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Station_ID    432 non-null    int64  
 1   Station_Name  432 non-null    object 
 2   Subway_lines  432 non-null    object 
 3   Boro          432 non-null    object 
 4   y_2014        432 non-null    int64  
 5   y_2015        432 non-null    int64  
 6   y_2016        432 non-null    int64  
 7   y_2017        432 non-null    int64  
 8   y_2018        432 non-null    int64  
 9   y_2019        432 non-null    int64  
 10  Chg           432 non-null    object 
 11  Pct_chg       432 non-null    object 
 12  Rank_2019     432 non-null    float64
 13  2019_Rank     432 non-null    int64  
dtypes: float64(1), int64(8), object(5)
memory usage: 50.6+ KB


/var/folders/vg/vb3rtmr10v34g75s975q03m80000gn/T/ipykernel_10253/3403491567.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ridership_df['y_2014'] = ridership_df.y_2014.astype(int)
/var/folders/vg/vb3rtmr10v34g75s975q03m80000gn/T/ipykernel_10253/3403491567.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ridership_df['y_2015'] = ridership_df.y_2015.astype(int)
/var/folders/vg/vb3rtmr10v34g75s975q03m80000gn/T/ipykernel_10253/3403491567.py:3: SettingWithCopyWarning: 
A value is trying to be set on a co

In [13]:
ridership_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 432 entries, 0 to 472
Data columns (total 14 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Station_ID    432 non-null    int64  
 1   Station_Name  432 non-null    object 
 2   Subway_lines  432 non-null    object 
 3   Boro          432 non-null    object 
 4   y_2014        432 non-null    int64  
 5   y_2015        432 non-null    int64  
 6   y_2016        432 non-null    int64  
 7   y_2017        432 non-null    int64  
 8   y_2018        432 non-null    int64  
 9   y_2019        432 non-null    int64  
 10  Chg           432 non-null    object 
 11  Pct_chg       432 non-null    object 
 12  Rank_2019     432 non-null    float64
 13  2019_Rank     432 non-null    int64  
dtypes: float64(1), int64(8), object(5)
memory usage: 50.6+ KB


In [ ]:
ridership_df.tail()

In [14]:
#Read in the MTA subway stations from json
import geopandas as gpd


In [15]:
cd '/Users/aaronwoodward/Desktop/Datasets/Cities/NYC/Subway Stations'

/Users/aaronwoodward/Desktop/Datasets/Cities/NYC/Subway Stations


In [16]:
stations_shapefiles = 'geo_export_10d03060-d569-40de-b528-c7b95b31aadd.shp'

In [17]:
stations_gdf = gpd.read_file(stations_shapefiles)
stations_gdf.head()

,line,name,notes,objectid,url,geometry
0,4-6-6 Express,Astor Pl,"4 nights, 6-all times, 6 Express-weekdays AM s...",1.0,http://web.mta.info/nyct/service/,POINT (-73.99107 40.73005)
1,4-6-6 Express,Canal St,"4 nights, 6-all times, 6 Express-weekdays AM s...",2.0,http://web.mta.info/nyct/service/,POINT (-74.00019 40.71880)
2,1-2,50th St,"1-all times, 2-nights",3.0,http://web.mta.info/nyct/service/,POINT (-73.98385 40.76173)
3,2-3-4,Bergen St,"4-nights, 3-all other times, 2-all times",4.0,http://web.mta.info/nyct/service/,POINT (-73.97500 40.68086)
4,3-4,Pennsylvania Ave,"4-nights, 3-all other times",5.0,http://web.mta.info/nyct/service/,POINT (-73.89489 40.66471)


In [18]:
#merging the 2 datasets
merged = stations_gdf.merge(ridership_df, left_on='objectid', right_on='Station_ID')

merged.head()

,line,name,notes,objectid,url,geometry,Station_ID,Station_Name,Subway_lines,Boro,y_2014,y_2015,y_2016,y_2017,y_2018,y_2019,Chg,Pct_chg,Rank_2019,2019_Rank
0,4-6-6 Express,Astor Pl,"4 nights, 6-all times, 6 Express-weekdays AM s...",1.0,http://web.mta.info/nyct/service/,POINT (-73.99107 40.73005),1,Astor Pl,6,Manhattan,17803,17274,16806,16377,16031,17180,"+1,149",+7.2%,84.0,84
1,4-6-6 Express,Canal St,"4 nights, 6-all times, 6 Express-weekdays AM s...",2.0,http://web.mta.info/nyct/service/,POINT (-74.00019 40.71880),2,Canal St,"J,N,Q,R,W,Z,6",Manhattan,44362,44409,45030,47601,47034,47629,+595,+1.3%,16.0,16
2,1-2,50th St,"1-all times, 2-nights",3.0,http://web.mta.info/nyct/service/,POINT (-73.98385 40.76173),3,50th St,1,Manhattan,27251,26927,26362,25932,26268,26712,+444,+1.7%,39.0,39
3,2-3-4,Bergen St,"4-nights, 3-all other times, 2-all times",4.0,http://web.mta.info/nyct/service/,POINT (-73.97500 40.68086),4,Bergen St,"2,3",Brooklyn,3923,3823,3884,3853,3653,3857,+204,+5.6%,334.0,334
4,3-4,Pennsylvania Ave,"4-nights, 3-all other times",5.0,http://web.mta.info/nyct/service/,POINT (-73.89489 40.66471),5,Pennsylvania Ave,3,Brooklyn,5718,6997,2485,3503,4331,4493,+162,+3.8%,310.0,310


In [19]:
merged.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Int64Index: 432 entries, 0 to 431
Data columns (total 20 columns):
 #   Column        Non-Null Count  Dtype   
---  ------        --------------  -----   
 0   line          432 non-null    object  
 1   name          432 non-null    object  
 2   notes         432 non-null    object  
 3   objectid      432 non-null    float64 
 4   url           432 non-null    object  
 5   geometry      432 non-null    geometry
 6   Station_ID    432 non-null    int64   
 7   Station_Name  432 non-null    object  
 8   Subway_lines  432 non-null    object  
 9   Boro          432 non-null    object  
 10  y_2014        432 non-null    int64   
 11  y_2015        432 non-null    int64   
 12  y_2016        432 non-null    int64   
 13  y_2017        432 non-null    int64   
 14  y_2018        432 non-null    int64   
 15  y_2019        432 non-null    int64   
 16  Chg           432 non-null    object  
 17  Pct_chg       432 non-null    object  
 18  Ra

In [20]:
merged = merged[['Station_ID',
                 'Station_Name',
                 'Subway_lines',
                 'Boro', 
                 'y_2014',
                 'y_2015',
                 'y_2016',
                 'y_2017',
                 'y_2018',
                 'y_2019',
                 'Chg',
                 'Pct_chg',
                 'Rank_2019',
                 'geometry'
                ]]

merged.head()

,Station_ID,Station_Name,Subway_lines,Boro,y_2014,y_2015,y_2016,y_2017,y_2018,y_2019,Chg,Pct_chg,Rank_2019,geometry
0,1,Astor Pl,6,Manhattan,17803,17274,16806,16377,16031,17180,"+1,149",+7.2%,84.0,POINT (-73.99107 40.73005)
1,2,Canal St,"J,N,Q,R,W,Z,6",Manhattan,44362,44409,45030,47601,47034,47629,+595,+1.3%,16.0,POINT (-74.00019 40.71880)
2,3,50th St,1,Manhattan,27251,26927,26362,25932,26268,26712,+444,+1.7%,39.0,POINT (-73.98385 40.76173)
3,4,Bergen St,"2,3",Brooklyn,3923,3823,3884,3853,3653,3857,+204,+5.6%,334.0,POINT (-73.97500 40.68086)
4,5,Pennsylvania Ave,3,Brooklyn,5718,6997,2485,3503,4331,4493,+162,+3.8%,310.0,POINT (-73.89489 40.66471)


In [21]:
def getPointCoords(row, geom, coord_type):
    """Calculates coordinates ('x' or 'y') of a Point geometry"""
    if coord_type == 'x':
        return row[geom].x
    elif coord_type == 'y':
        return row[geom].y

In [22]:
# Calculate x coordinates
merged['x'] = merged.apply(getPointCoords, geom='geometry', coord_type='x', axis=1)

# Calculate y coordinates
merged['y'] = merged.apply(getPointCoords, geom='geometry', coord_type='y', axis=1)

# Let's see what we have now
merged.head()

,Station_ID,Station_Name,Subway_lines,Boro,y_2014,y_2015,y_2016,y_2017,y_2018,y_2019,Chg,Pct_chg,Rank_2019,geometry,x,y
0,1,Astor Pl,6,Manhattan,17803,17274,16806,16377,16031,17180,"+1,149",+7.2%,84.0,POINT (-73.99107 40.73005),-73.991070,40.730054
1,2,Canal St,"J,N,Q,R,W,Z,6",Manhattan,44362,44409,45030,47601,47034,47629,+595,+1.3%,16.0,POINT (-74.00019 40.71880),-74.000193,40.718803
2,3,50th St,1,Manhattan,27251,26927,26362,25932,26268,26712,+444,+1.7%,39.0,POINT (-73.98385 40.76173),-73.983849,40.761728
3,4,Bergen St,"2,3",Brooklyn,3923,3823,3884,3853,3653,3857,+204,+5.6%,334.0,POINT (-73.97500 40.68086),-73.974999,40.680862
4,5,Pennsylvania Ave,3,Brooklyn,5718,6997,2485,3503,4331,4493,+162,+3.8%,310.0,POINT (-73.89489 40.66471),-73.894886,40.664714


In [23]:
# Make a copy and drop the geometry column
df = merged.drop('geometry', axis=1).copy()

In [24]:
df.head()

,Station_ID,Station_Name,Subway_lines,Boro,y_2014,y_2015,y_2016,y_2017,y_2018,y_2019,Chg,Pct_chg,Rank_2019,x,y
0,1,Astor Pl,6,Manhattan,17803,17274,16806,16377,16031,17180,"+1,149",+7.2%,84.0,-73.991070,40.730054
1,2,Canal St,"J,N,Q,R,W,Z,6",Manhattan,44362,44409,45030,47601,47034,47629,+595,+1.3%,16.0,-74.000193,40.718803
2,3,50th St,1,Manhattan,27251,26927,26362,25932,26268,26712,+444,+1.7%,39.0,-73.983849,40.761728
3,4,Bergen St,"2,3",Brooklyn,3923,3823,3884,3853,3653,3857,+204,+5.6%,334.0,-73.974999,40.680862
4,5,Pennsylvania Ave,3,Brooklyn,5718,6997,2485,3503,4331,4493,+162,+3.8%,310.0,-73.894886,40.664714


In [25]:
df['Location'] = "(" + df['y'].astype(str) + ", " + df['x'].astype(str) +")"

In [26]:
df.head()

,Station_ID,Station_Name,Subway_lines,Boro,y_2014,y_2015,y_2016,y_2017,y_2018,y_2019,Chg,Pct_chg,Rank_2019,x,y,Location
0,1,Astor Pl,6,Manhattan,17803,17274,16806,16377,16031,17180,"+1,149",+7.2%,84.0,-73.991070,40.730054,"(40.73005400028978, -73.99106999861966)"
1,2,Canal St,"J,N,Q,R,W,Z,6",Manhattan,44362,44409,45030,47601,47034,47629,+595,+1.3%,16.0,-74.000193,40.718803,"(40.71880300107709, -74.00019299927328)"
2,3,50th St,1,Manhattan,27251,26927,26362,25932,26268,26712,+444,+1.7%,39.0,-73.983849,40.761728,"(40.76172799961419, -73.98384899986625)"
3,4,Bergen St,"2,3",Brooklyn,3923,3823,3884,3853,3653,3857,+204,+5.6%,334.0,-73.974999,40.680862,"(40.68086213682956, -73.97499915116808)"
4,5,Pennsylvania Ave,3,Brooklyn,5718,6997,2485,3503,4331,4493,+162,+3.8%,310.0,-73.894886,40.664714,"(40.66471445143568, -73.89488591154061)"


In [27]:
df.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Int64Index: 432 entries, 0 to 431
Data columns (total 16 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Station_ID    432 non-null    int64  
 1   Station_Name  432 non-null    object 
 2   Subway_lines  432 non-null    object 
 3   Boro          432 non-null    object 
 4   y_2014        432 non-null    int64  
 5   y_2015        432 non-null    int64  
 6   y_2016        432 non-null    int64  
 7   y_2017        432 non-null    int64  
 8   y_2018        432 non-null    int64  
 9   y_2019        432 non-null    int64  
 10  Chg           432 non-null    object 
 11  Pct_chg       432 non-null    object 
 12  Rank_2019     432 non-null    float64
 13  x             432 non-null    float64
 14  y             432 non-null    float64
 15  Location      432 non-null    object 
dtypes: float64(3), int64(7), object(6)
memory usage: 57.4+ KB


In [28]:
import math
from ast import literal_eval
def merc(Coords):
    Coordinates = literal_eval(Coords)
    lat = Coordinates[0]
    lon = Coordinates[1]
    
    r_major = 6378137.000
    x = r_major * math.radians(lon)
    scale = x/lon
    y = 180.0/math.pi * math.log(math.tan(math.pi/4.0 + 
        lat * (math.pi/180.0)/2.0)) * scale
    return (x, y)

In [29]:
merc('(40.730054, -73.991070)')

(-8236648.23564946, 4972605.8591247015)

In [30]:
df['coords_x'] = df['Location'].apply(lambda x: merc(x)[0])
df['coords_y'] = df['Location'].apply(lambda x: merc(x)[1])

In [31]:
df.head()

,Station_ID,Station_Name,Subway_lines,Boro,y_2014,y_2015,y_2016,y_2017,y_2018,y_2019,Chg,Pct_chg,Rank_2019,x,y,Location,coords_x,coords_y
0,1,Astor Pl,6,Manhattan,17803,17274,16806,16377,16031,17180,"+1,149",+7.2%,84.0,-73.991070,40.730054,"(40.73005400028978, -73.99106999861966)",-8.236648e+06,4.972606e+06
1,2,Canal St,"J,N,Q,R,W,Z,6",Manhattan,44362,44409,45030,47601,47034,47629,+595,+1.3%,16.0,-74.000193,40.718803,"(40.71880300107709, -74.00019299927328)",-8.237664e+06,4.970953e+06
2,3,50th St,1,Manhattan,27251,26927,26362,25932,26268,26712,+444,+1.7%,39.0,-73.983849,40.761728,"(40.76172799961419, -73.98384899986625)",-8.235844e+06,4.977260e+06
3,4,Bergen St,"2,3",Brooklyn,3923,3823,3884,3853,3653,3857,+204,+5.6%,334.0,-73.974999,40.680862,"(40.68086213682956, -73.97499915116808)",-8.234859e+06,4.965382e+06
4,5,Pennsylvania Ave,3,Brooklyn,5718,6997,2485,3503,4331,4493,+162,+3.8%,310.0,-73.894886,40.664714,"(40.66471445143568, -73.89488591154061)",-8.225941e+06,4.963012e+06


In [32]:
p = figure(x_range=(-8237000, -8200000), y_range=(4980000, 4950000),
           x_axis_type="mercator", y_axis_type="mercator")

tile_provider = get_provider(Vendors.CARTODBPOSITRON)
p.add_tile(tile_provider)
show(p)

In [33]:
p = figure(x_axis_type="mercator", y_axis_type="mercator")

tile_provider = get_provider(Vendors.CARTODBPOSITRON)
p.add_tile(tile_provider)
#p.add_tile(CARTODBPOSITRON)

p.circle(x = 'coords_x',
         y = 'coords_y',
         source = df)
output_notebook()
show(p)

Loading BokehJS ...

In [ ]:
#import matplotlib as mpl

In [34]:
from bokeh.io import output_notebook, show, output_file
from bokeh.models import ColumnDataSource, HoverTool, EqHistColorMapper, ColorBar
from bokeh.palettes import brewer
from bokeh.palettes import Magma, Inferno, Plasma, Viridis

source = ColumnDataSource(data=dict(
                        x=list(df['coords_x']), 
                        y=list(df['coords_y']),
                        y_2014 = list(df['y_2014']),
                        y_2015 = list(df['y_2015']),
                        y_2016 = list(df['y_2016']),
                        y_2017 = list(df['y_2017']),
                        y_2018 = list(df['y_2018']),
                        y_2019 = list(df['y_2019']),
                        Station = list(df['Station_Name']),
                        Subway_Lines=list(df['Subway_lines']),
                        Change=list(df['Chg']),
                        Pct_chg_2019=list(df['Pct_chg']),
                        Rank_2019=list(df['Rank_2019'])))

hover = HoverTool(tooltips=[
    ("Station", "@Station"),
    ("Subway Lines", "@Subway_Lines"),
    ("2019", "@{y_2019}{0,0}"),
    ("2018", "@{y_2018}{0,0}"),
    ("2017", "@{y_2017}{0,0}"),
    ("2016", "@{y_2016}{0,0}"),
    ("2015", "@{y_2015}{0,0}"),
    ("2014", "@{y_2014}{0,0}"),
    ("Actual change from 2018", "@Change"),
    ("% change from 2018", "@Pct_chg_2019"),
    ("Rank", "@Rank_2019")
])

p = figure(title = 'Average Weekday Subway Ridership, 2019',
           x_axis_type="mercator",
           y_axis_type="mercator",
           tools=[hover, 'wheel_zoom']
          )

tile_provider = get_provider(Vendors.CARTODBPOSITRON)
p.add_tile(tile_provider)

#Define a sequential multi-hue color palette.
#palette = brewer['Plasma'][8]
#palette = mpl['Plasma'][4]
#palette = Plasma[4]
palette = Magma[11]

#Reverse color order so that dark purple/magenta is to signal high activity.
palette = palette[::-1]

#exp_cmap = LinearColorMapper(palette=palette,
#                            low = min(df['y_2019']),
#                            high = max(df['y_2019']))

exp_cmap = EqHistColorMapper(palette=palette, low=min(df['y_2019']), high=max(df['y_2019']))

p.circle(x='x',
         y='y',
         source=source,
         size=6,
         line_color="black",
         fill_color={"field":"y_2019", "transform":exp_cmap},
         alpha=1
        )

bar = ColorBar(color_mapper=exp_cmap, location=(0,0))
#bar.formatter = NumeralTickFormatter(format'0,0')
p.add_layout(bar, 'left')

output_file("NYC_Subway_Ridership_Map_2019.html")

show(p)

In [35]:
cd '/Users/aaronwoodward/Desktop/Datasets/Cities/NYC/Subway Lines'

/Users/aaronwoodward/Desktop/Datasets/Cities/NYC/Subway Lines


In [36]:
lines_shapefiles = 'geo_export_5cda2676-7bd2-492e-b02b-e69dd962562f.shp'

In [37]:
lines_gdf = gpd.read_file(lines_shapefiles)
lines_gdf.head()

,id,name,objectid,rt_symbol,shape_len,url,geometry
0,2000393.0,G,753.0,G,2438.200249,http://web.mta.info/nyct/service/,"LINESTRING (-73.99488 40.68020, -73.99427 40.6..."
1,2000394.0,G,754.0,G,3872.834411,http://web.mta.info/nyct/service/,"LINESTRING (-73.97958 40.65993, -73.97966 40.6..."
2,2000469.0,Q,755.0,N,1843.366331,http://web.mta.info/nyct/service/,"LINESTRING (-73.97586 40.57597, -73.97654 40.5..."
3,2000294.0,M,756.0,B,1919.559203,http://web.mta.info/nyct/service/,"LINESTRING (-73.92414 40.75229, -73.92405 40.7..."
4,2000296.0,M,757.0,B,2385.698536,http://web.mta.info/nyct/service/,"LINESTRING (-73.91345 40.75617, -73.90905 40.7..."


In [38]:
def line_color(lines_gdf):
    if(lines_gdf['name']=='1') or (lines_gdf['name']=='1-2-3') or (lines_gdf['name']=='2-3') or (lines_gdf['name']=='2') or (lines_gdf['name']=='3'):
        return '#f20404'
    elif(lines_gdf['name']=='4') or (lines_gdf['name']=='5') or (lines_gdf['name']=='6') or (lines_gdf['name']=='4-5') or (lines_gdf['name']=='4-5-6'):
        return '#2fbf03'
    elif(lines_gdf['name']=='7'):
        return '#8c00a7'
    elif(lines_gdf['name']=='A') or (lines_gdf['name']=='A-C') or (lines_gdf['name']=='E') or (lines_gdf['name']=='A-C-E'):
        return '#024ed0'
    elif(lines_gdf['name']=='B') or (lines_gdf['name']=='D') or (lines_gdf['name']=='B-D') or (lines_gdf['name']=='F') or (lines_gdf['name']=='M') or (lines_gdf['name']=='F-M') or (lines_gdf['name']=='B-D-F-M'):
        return '#f08b0e'
    elif(lines_gdf['name']=='G'):
        return '#6be12f'
    elif(lines_gdf['name']=='J-Z'):
        return '#ae5913'
    elif(lines_gdf['name']=='L'):
        return '#9c9894'
    elif(lines_gdf['name']=='S'):
        return '#5a5952'
    else:
        return '#f6d50a'   

In [39]:
lines_gdf['color'] = lines_gdf.apply(line_color, axis = 1)

In [56]:
#lines_gdf['rt_symbol'].unique()

array(['G', 'N', 'B', 'L', 'A', '7', 'J', '1', '4'], dtype=object)

In [57]:
#lines_gdf['name'].unique()

array(['G', 'Q', 'M', 'S', 'A', 'B-D', 'B-D-F-M', 'R', 'N-Q-R', 'N-Q',
       'N-R', 'F', 'F-M', 'E', '7', 'J-Z', 'L', 'A-C', 'D', '1-2-3', 'B',
       '4-5-6', 'N', '1', 'N-W', '2-3', '2', '4-5', '5', '4', '3',
       'A-C-E', 'N-Q-R-W', 'N-R-W', '6', 'R-W'], dtype=object)

In [40]:
lines_gdf.head(10)

,id,name,objectid,rt_symbol,shape_len,url,geometry,color
0,2000393.0,G,753.0,G,2438.200249,http://web.mta.info/nyct/service/,"LINESTRING (-73.99488 40.68020, -73.99427 40.6...",#6be12f
1,2000394.0,G,754.0,G,3872.834411,http://web.mta.info/nyct/service/,"LINESTRING (-73.97958 40.65993, -73.97966 40.6...",#6be12f
2,2000469.0,Q,755.0,N,1843.366331,http://web.mta.info/nyct/service/,"LINESTRING (-73.97586 40.57597, -73.97654 40.5...",#f6d50a
3,2000294.0,M,756.0,B,1919.559203,http://web.mta.info/nyct/service/,"LINESTRING (-73.92414 40.75229, -73.92405 40.7...",#f08b0e
4,2000296.0,M,757.0,B,2385.698536,http://web.mta.info/nyct/service/,"LINESTRING (-73.91345 40.75617, -73.90905 40.7...",#f08b0e
5,2000295.0,M,758.0,B,2470.818582,http://web.mta.info/nyct/service/,"LINESTRING (-73.92056 40.75679, -73.91967 40.7...",#f08b0e
6,2000239.0,G,759.0,G,2448.069200,http://web.mta.info/nyct/service/,"LINESTRING (-73.98510 40.68834, -73.98415 40.6...",#6be12f
7,2000238.0,G,760.0,G,690.126468,http://web.mta.info/nyct/service/,"LINESTRING (-73.98510 40.68835, -73.98732 40.6...",#6be12f
8,2000277.0,S,761.0,L,2097.717962,http://web.mta.info/nyct/service/,"LINESTRING (-73.82762 40.58333, -73.82284 40.5...",#5a5952
9,2000265.0,A,762.0,A,5840.912038,http://web.mta.info/nyct/service/,"LINESTRING (-73.80922 40.59318, -73.81545 40.6...",#024ed0


In [41]:
def getLineCoords(row, geom, coord_type):
    """Returns a list of coordinates ('x' or 'y') of a LineString geometry"""
    if coord_type == 'x':
        return list( row[geom].coords.xy[0] )
    elif coord_type == 'y':
        return list( row[geom].coords.xy[1] )


In [42]:
# Calculate x coordinates of the line
lines_gdf['x'] = lines_gdf.apply(getLineCoords, geom='geometry', coord_type='x', axis=1)

# Calculate y coordinates of the line
lines_gdf['y'] = lines_gdf.apply(getLineCoords, geom='geometry', coord_type='y', axis=1)

# Let's see what we have now
lines_gdf.head()

,id,name,objectid,rt_symbol,shape_len,url,geometry,color,x,y
0,2000393.0,G,753.0,G,2438.200249,http://web.mta.info/nyct/service/,"LINESTRING (-73.99488 40.68020, -73.99427 40.6...",#6be12f,"[-73.99487524803018, -73.99427469414127, -73.9...","[40.6802035460625, 40.68081016270495, 40.68088..."
1,2000394.0,G,754.0,G,3872.834411,http://web.mta.info/nyct/service/,"LINESTRING (-73.97958 40.65993, -73.97966 40.6...",#6be12f,"[-73.97957543205142, -73.97965516869557, -73.9...","[40.65993069553076, 40.65981741117469, 40.6588..."
2,2000469.0,Q,755.0,N,1843.366331,http://web.mta.info/nyct/service/,"LINESTRING (-73.97586 40.57597, -73.97654 40.5...",#f6d50a,"[-73.97585637503069, -73.97654226924728, -73.9...","[40.5759745053941, 40.57575795972222, 40.57572..."
3,2000294.0,M,756.0,B,1919.559203,http://web.mta.info/nyct/service/,"LINESTRING (-73.92414 40.75229, -73.92405 40.7...",#f08b0e,"[-73.92414355434533, -73.9240493608916, -73.92...","[40.752290926571746, 40.75235476981202, 40.752..."
4,2000296.0,M,757.0,B,2385.698536,http://web.mta.info/nyct/service/,"LINESTRING (-73.91345 40.75617, -73.90905 40.7...",#f08b0e,"[-73.91344685471373, -73.90904942550716, -73.9...","[40.75617157636837, 40.75410484686675, 40.7530..."


In [43]:
# Make a copy and drop the geometry column
lines_df = lines_gdf.drop('geometry', axis=1).copy()

# Point DataSource
lines_source = ColumnDataSource(lines_df)

In [44]:
color = lines_df['color']

In [71]:
#lines_df['Route'] = "(" + lines_df['y'].astype(str) + ", " + lines_df['x'].astype(str) +")"

In [45]:
# Initialize our plot figure
p = figure(title="A map of the MTA New York City Subway System")

tile_provider = get_provider(Vendors.CARTODBPOSITRON)
p.add_tile(tile_provider)

# Add the lines to the map from our 'msource' ColumnDataSource -object
p.multi_line('x', 'y', source=lines_source, color='color', line_width=2)

# Output filepath
output_file("NYC_Subway_System_Map.html")

# Save the map
show(p)

In [ ]:
#Line Colors:
    #1,2,3 = #f20404
    #4,5,6 = #2fbf03
    #7 = #8c00a7
    #A,C,E = #024ed0
    #B,D,F,M = #f08b0e
    #G = #6be12f
    #J,Z = #ae5913
    #L = #9c9894
    #N,Q,R,W = #f6d50a


In [ ]:
###Mercator conversion

In [46]:
lines_shapefiles = 'geo_export_5cda2676-7bd2-492e-b02b-e69dd962562f.shp'

In [47]:
lines_merc_gdf = gpd.read_file(lines_shapefiles)

#https://gis.stackexchange.com/questions/346181/project-large-collection-of-shapely-object-to-different-crs
merc_conv = lines_merc_gdf['geometry'].to_crs("epsg:3857")
lines_merc_gdf['merc_coords'] = merc_conv
#gdf_wgs84.to_file("output.geojson", driver="GeoJSON")

lines_merc_gdf.head()

,id,name,objectid,rt_symbol,shape_len,url,geometry,merc_coords
0,2000393.0,G,753.0,G,2438.200249,http://web.mta.info/nyct/service/,"LINESTRING (-73.99488 40.68020, -73.99427 40.6...","LINESTRING (-8237071.834 4965285.580, -8237004..."
1,2000394.0,G,754.0,G,3872.834411,http://web.mta.info/nyct/service/,"LINESTRING (-73.97958 40.65993, -73.97966 40.6...","LINESTRING (-8235368.666 4962310.184, -8235377..."
2,2000469.0,Q,755.0,N,1843.366331,http://web.mta.info/nyct/service/,"LINESTRING (-73.97586 40.57597, -73.97654 40.5...","LINESTRING (-8234954.663 4949997.756, -8235031..."
3,2000294.0,M,756.0,B,1919.559203,http://web.mta.info/nyct/service/,"LINESTRING (-73.92414 40.75229, -73.92405 40.7...","LINESTRING (-8229198.018 4975873.004, -8229187..."
4,2000296.0,M,757.0,B,2385.698536,http://web.mta.info/nyct/service/,"LINESTRING (-73.91345 40.75617, -73.90905 40.7...","LINESTRING (-8228007.267 4976443.278, -8227517..."


In [48]:
def getLineCoords(row, geom, coord_type):
    """Returns a list of coordinates ('x' or 'y') of a LineString geometry"""
    if coord_type == 'x':
        return list( row[geom].coords.xy[0] )
    elif coord_type == 'y':
        return list( row[geom].coords.xy[1] )


In [49]:
# Calculate x coordinates of the line
lines_merc_gdf['x'] = lines_merc_gdf.apply(getLineCoords, geom='merc_coords', coord_type='x', axis=1)

# Calculate y coordinates of the line
lines_merc_gdf['y'] = lines_merc_gdf.apply(getLineCoords, geom='merc_coords', coord_type='y', axis=1)

# Let's see what we have now
lines_merc_gdf.head()

,id,name,objectid,rt_symbol,shape_len,url,geometry,merc_coords,x,y
0,2000393.0,G,753.0,G,2438.200249,http://web.mta.info/nyct/service/,"LINESTRING (-73.99488 40.68020, -73.99427 40.6...","LINESTRING (-8237071.834 4965285.580, -8237004...","[-8237071.833922522, -8237004.980569415, -8236...","[4965285.580068259, 4965374.625638519, 4965385..."
1,2000394.0,G,754.0,G,3872.834411,http://web.mta.info/nyct/service/,"LINESTRING (-73.97958 40.65993, -73.97966 40.6...","LINESTRING (-8235368.666 4962310.184, -8235377...","[-8235368.666198536, -8235377.54244116, -82354...","[4962310.183773335, 4962293.559848732, 4962153..."
2,2000469.0,Q,755.0,N,1843.366331,http://web.mta.info/nyct/service/,"LINESTRING (-73.97586 40.57597, -73.97654 40.5...","LINESTRING (-8234954.663 4949997.756, -8235031...","[-8234954.662664757, -8235031.0160596855, -823...","[4949997.755558339, 4949966.0184754, 4949961.7..."
3,2000294.0,M,756.0,B,1919.559203,http://web.mta.info/nyct/service/,"LINESTRING (-73.92414 40.75229, -73.92405 40.7...","LINESTRING (-8229198.018 4975873.004, -8229187...","[-8229198.01779858, -8229187.532231273, -82291...","[4975873.004364504, 4975882.386061685, 4975895..."
4,2000296.0,M,757.0,B,2385.698536,http://web.mta.info/nyct/service/,"LINESTRING (-73.91345 40.75617, -73.90905 40.7...","LINESTRING (-8228007.267 4976443.278, -8227517...","[-8228007.26664242, -8227517.747062346, -82272...","[4976443.278149745, 4976139.561556996, 4975988..."


In [ ]:
#Applying the appropriate colors (color of the lines that you see on NYC Subway Map) to the subway lines

In [50]:
def line_color(lines_merc_gdf):
    if(lines_merc_gdf['name']=='1') or (lines_merc_gdf['name']=='1-2-3') or (lines_merc_gdf['name']=='2-3') or (lines_merc_gdf['name']=='2') or (lines_merc_gdf['name']=='3'):
        return '#f20404'
    elif(lines_merc_gdf['name']=='4') or (lines_merc_gdf['name']=='5') or (lines_merc_gdf['name']=='6') or (lines_merc_gdf['name']=='4-5') or (lines_merc_gdf['name']=='4-5-6'):
        return '#2fbf03'
    elif(lines_merc_gdf['name']=='7'):
        return '#8c00a7'
    elif(lines_merc_gdf['name']=='A') or (lines_merc_gdf['name']=='A-C') or (lines_merc_gdf['name']=='E') or (lines_merc_gdf['name']=='A-C-E'):
        return '#024ed0'
    elif(lines_merc_gdf['name']=='B') or (lines_merc_gdf['name']=='D') or (lines_merc_gdf['name']=='B-D') or (lines_merc_gdf['name']=='F') or (lines_merc_gdf['name']=='M') or (lines_merc_gdf['name']=='F-M') or (lines_merc_gdf['name']=='B-D-F-M'):
        return '#f08b0e'
    elif(lines_merc_gdf['name']=='G'):
        return '#6be12f'
    elif(lines_merc_gdf['name']=='J-Z'):
        return '#ae5913'
    elif(lines_merc_gdf['name']=='L'):
        return '#9c9894'
    elif(lines_merc_gdf['name']=='S'):
        return '#5a5952'
    else:
        return '#f6d50a'   

AttributeError: 'GeoDataFrame' object has no attribute 'unique'

In [51]:
lines_merc_gdf['color'] = lines_merc_gdf.apply(line_color, axis = 1)

In [52]:
# Make a copy and drop the geometry column
lines_merc_df = lines_merc_gdf.drop(['geometry','merc_coords'], axis=1).copy()

# Point DataSource
lines_merc_source = ColumnDataSource(lines_merc_df)

In [53]:
color = lines_merc_df['color']

In [54]:
# Initialize our plot figure
p = figure(title="A map of the MTA New York City Subway System")

tile_provider = get_provider(Vendors.CARTODBPOSITRON)
p.add_tile(tile_provider)

# Add the lines to the map from our 'msource' ColumnDataSource -object
p.multi_line('x', 'y', source=lines_merc_source, color='color', line_width=2)

# Output filepath
output_file("NYC_Subway_System_Map.html")

# Save the map
show(p)

In [ ]:
###Putting it all together###

In [55]:
#Add stations and coloring by ridership by station
source = ColumnDataSource(data=dict(
                        x=list(df['coords_x']), 
                        y=list(df['coords_y']),
                        y_2014 = list(df['y_2014']),
                        y_2015 = list(df['y_2015']),
                        y_2016 = list(df['y_2016']),
                        y_2017 = list(df['y_2017']),
                        y_2018 = list(df['y_2018']),
                        y_2019 = list(df['y_2019']),
                        Station = list(df['Station_Name']),
                        Subway_Lines=list(df['Subway_lines']),
                        Change=list(df['Chg']),
                        Pct_chg_2019=list(df['Pct_chg']),
                        Rank_2019=list(df['Rank_2019'])))

hover = HoverTool(tooltips=[
    ("Station", "@Station"),
    ("Subway Lines", "@Subway_Lines"),
    ("2019", "@{y_2019}{0,0}"),
    ("2018", "@{y_2018}{0,0}"),
    ("2017", "@{y_2017}{0,0}"),
    ("2016", "@{y_2016}{0,0}"),
    ("2015", "@{y_2015}{0,0}"),
    ("2014", "@{y_2014}{0,0}"),
    ("Actual change from 2018", "@Change"),
    ("% change from 2018", "@Pct_chg_2019"),
    ("Rank", "@Rank_2019")
])

p = figure(title = 'Average Weekday Subway Ridership, 2019',
           x_axis_type="mercator",
           y_axis_type="mercator",
           tools=[hover, 'wheel_zoom']
          )

#tile_provider = get_provider(Vendors.CARTODBPOSITRON)
#p.add_tile(tile_provider)

#Define a sequential multi-hue color palette.
#palette = brewer['Plasma'][8]
#palette = mpl['Plasma'][4]
#palette = Plasma[4]
palette = Magma[11]

#Reverse color order so that dark purple/magenta is to signal high activity.
palette = palette[::-1]

#exp_cmap = LinearColorMapper(palette=palette,
#                            low = min(df['y_2019']),
#                            high = max(df['y_2019']))

exp_cmap = EqHistColorMapper(palette=palette, low=min(df['y_2019']), high=max(df['y_2019']))

#p.circle(x='x',
#         y='y',
#         source=source,
#         size=6,
#         line_color="black",
#         fill_color={"field":"y_2019", "transform":exp_cmap},
#        alpha=1
#        )

bar = ColorBar(color_mapper=exp_cmap, location=(0,0))
#bar.formatter = NumeralTickFormatter(format'0,0')
p.add_layout(bar, 'left')


# Output filepath
#output_file("NYC_Subway_System_Map.html")

# Save the map
#show(p)

In [64]:
# Initialize our plot figure
p = figure(title="MTA New York City Subway System and Pre-pandemic Average Weekday Ridership",
          tools=[hover,'wheel_zoom', 'pan'])

tile_provider = get_provider(Vendors.CARTODBPOSITRON)
p.add_tile(tile_provider)

# Add the lines to the map from our 'msource' ColumnDataSource -object
p.multi_line('x', 'y', source=lines_merc_source, color='color', line_width=3)

p.circle(x='x',
         y='y',
         source=source,
         size=6,
         line_color="black",
         fill_color={"field":"y_2019", "transform":exp_cmap},
         alpha=1
        )
p.add_layout(bar, 'right')


show(p)

In [65]:
line_list = ['1','2','3','2-3','1-2-3','4','5','6','4-5','4-5-6','7','A','A-C','A-C-E','B','D','B-D','F','M','F-M',
            'B-D-F-M','G','J-Z','L','S','N','R','Q','W','R-W','N-Q','N-R-W','N-Q-R-W','N-W']

In [70]:
from bokeh.io import output_file, show
from bokeh.layouts import widgetbox
from bokeh.models.widgets import Dropdown

output_file("dropdown_menu.html")
 
menu = line_list
dropdown_menu = Dropdown(label = "Subway Lines",
#                         button_type = "warning",
                         menu = menu)
 
show(widgetbox(dropdown_menu))


In [74]:
# Initialize our plot figure
p = figure(title="MTA New York City Subway System and Pre-pandemic Average Weekday Ridership",
          tools=[hover,'wheel_zoom', 'pan'])

tile_provider = get_provider(Vendors.CARTODBPOSITRON)
p.add_tile(tile_provider)

# Add the lines to the map from our 'msource' ColumnDataSource -object
p.multi_line('x', 'y', source=lines_merc_source, color='color', line_width=3)

p.circle(x='x',
         y='y',
         source=source,
         size=6,
         line_color="black",
         fill_color={"field":"y_2019", "transform":exp_cmap},
         alpha=1
        )


menu = line_list
dropdown_menu = Dropdown(label = "Subway Lines",
#                         button_type = "warning",
                         menu = menu)
p.add_layout(bar, 'right')

show(widgetbox(dropdown_menu))
show(p)

Error: could not locate runnable browser

In [131]:
from bokeh.models.widgets import CheckboxGroup
# Create the checkbox selection element, available carriers is a  
# list of all airlines in the data
line_selection = CheckboxGroup(labels=line_list,
                               active = [0, 1])

In [ ]:
### END OF FILE ###

In [74]:
lines_df['coords_x'] = lines_df['Route'].apply(lambda x: merc(x)[0])
lines_df['coords_y'] = lines_df['Route'].apply(lambda x: merc(x)[1])

TypeError: must be real number, not list

In [97]:
def convertCoordinates(linestring, transformer):
    projected = shapely.ops.transform(transformer.transform, linestring)
    return projected

transformer = pyproj.Transformer.from_proj(pyproj.Proj(init='epsg:4326'), pyproj.Proj(init='epsg:3857'))
for linestring in linestrings:
    projected_coordinates = convertCoordinates(linestring, transformer)

/opt/anaconda3/envs/geo_env/lib/python3.9/site-packages/pyproj/crs/crs.py:68: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))
/opt/anaconda3/envs/geo_env/lib/python3.9/site-packages/pyproj/crs/crs.py:306: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  projstring = _prepare_from_string(" ".join((projstring, projkwargs)))
/opt/anaconda3/envs/geo_env/lib/python3.9/site-packages/pyproj/crs/crs.py:68: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initializatio

NameError: name 'linestrings' is not defined

In [72]:
import math
from ast import literal_eval
def merc(Coords):
    Coordinates = literal_eval(Coords)
    lat = Coordinates[0]
    lon = Coordinates[1]
    
    r_major = 6378137.000
    x = r_major * math.radians(lon)
    scale = x/lon
    y = 180.0/math.pi * math.log(math.tan(math.pi/4.0 + 
        lat * (math.pi/180.0)/2.0)) * scale
    return (x, y)

In [92]:
#https://gis.stackexchange.com/questions/127427/transforming-shapely-polygon-and-multipolygon-objects

import pyproj
from shapely.ops import transform

project = pyproj.Transformer.from_proj(
    pyproj.Proj(init='epsg:4326'), # source coordinate system
    pyproj.Proj(init='epsg:3857')) # destination coordinate system

# g1 is a shapley Polygon

merc_conv = transform(project.transform, lines_gdf['geometry'])  # apply projection

/opt/anaconda3/envs/geo_env/lib/python3.9/site-packages/pyproj/crs/crs.py:68: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))
/opt/anaconda3/envs/geo_env/lib/python3.9/site-packages/pyproj/crs/crs.py:306: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  projstring = _prepare_from_string(" ".join((projstring, projkwargs)))
/opt/anaconda3/envs/geo_env/lib/python3.9/site-packages/pyproj/crs/crs.py:68: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initializatio

ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

In [86]:
#https://gis.stackexchange.com/questions/78838/converting-projected-coordinates-to-lat-lon-using-python

from pyproj import Proj, transform

#inProj = Proj(init='epsg:3857')
#outProj = Proj(init='epsg:4326')
#x1,y1 = -11705274.6374,4826473.6922
#x2,y2 = transform(inProj,outProj,x1,y1)

inProj = Proj(init='epsg:4326')
outProj = Proj(init='epsg:3857')
#x1,y1 = -11705274.6374,4826473.6922
#x2,y2 = transform(inProj,outProj,x1,y1)
merc_conv = transform(inProj,outProj,lines_df['x'],lines_df['y'])


/opt/anaconda3/envs/geo_env/lib/python3.9/site-packages/pyproj/crs/crs.py:68: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))
/opt/anaconda3/envs/geo_env/lib/python3.9/site-packages/pyproj/crs/crs.py:306: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  projstring = _prepare_from_string(" ".join((projstring, projkwargs)))
/opt/anaconda3/envs/geo_env/lib/python3.9/site-packages/pyproj/crs/crs.py:68: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initializatio

TypeError: 'module' object is not callable